
# Introduction

> * Explain why pay-for-blind-signature needs additional attension
> * mention that it is not included in [Nadav Kohen's list](https://lists.linuxfoundation.org/pipermail/lightning-dev/2020-April/002647.html)




## Preliminaries

We use [Camenisch and Stadler style notation](https://crypto.ethz.ch/publications/files/CamSta97b.pdf) to describe Zero knowledge Proof of Knowledge (ZKPoK), i.e.

$$
PK\{(x, y): \verb|statements  about | x, y ...\}
$$

### Variables

* $Hash$ ... Hash function
* $Info$ ... Data previously shared with Buyer and the Seller
* $X := x * G$ ... private and public key for the seller.
* $G$, $H$ ... Generators. $G$ is the basepoint in secp256k1

:# Pay for blind signature

If you understand both pay-for-signature scheme for PTLC lightning and Schnorr blind signature protocol, there's not much surprise here. I recommend Nadav Kohen's explaination for recap.

* [Schnorr Blind signature](https://suredbits.com/schnorr-applications-blind-signatures/)
* [Pay for signature](https://suredbits.com/payment-points-part-4-selling-signatures/)


## Actual protocol
Seller:
* creates ranodm Nonce $R := k * G$ and sends it to the Buyer

Buyer:
* Gets random blinding factor $t_1$, $t_2$, $t_3$
* computes following items
  * blinded pubkey $X' = X + t_1$
  * blinded nonce ... $R' := R + t_2 * G + t_3 * X$
  * challenge ... $c = Hash(X' || R' || m)$
  * blinded challenge ... $c' = c + t_3$
* Make payment offer for a blinded signature $s'$ and send $c'$
  * $s' * G = R + c' * X$

Seller:
* With some predefined probability, deny the payment offer and restart everything from the beggining. (To avoid the Wagner's attack, see below.)
* Otherwise
  * computes $s = k + c' * x$
  * Receive payment by revaling $s'$

Buyer:
* Unblinds $s'$
  * $s = s' + t_2 + c' * t_1$
  * $s = (k + t_2 + x * t_3) + Hash((x + t_1) * G || (k + t_2 + x * t_3) * G || m)(x + t_1) $
  * So the $s$ will make valid signature for pubkey $X + t_1 * G$ with nonce $(k + t_2 + x * t_3) * G$




# Blind submarine-swap

> TODO: explain

# Pay for clause blind signature

The regular blind signature described above is amenable to the [Wagner's attack](https://www.iacr.org/archive/crypto2002/24420288/24420288.pdf). Luckly, [Fuchsbauer et al has proposed the simple fix](https://eprint.iacr.org/2019/877.pdf) to make it secure.

Incorporating the fix works just fine in our LN settings, but with one caveat, the Buyer must make two payment offer to the Seller and just one gets solved, thus wasting the channel liquidity needlessly.
We can avoid this if we can use different base points for the signature and those used in secp256k1 (i.e. LN)

In this case, we don't need public key blinding.
(TODO: explain why)


## Actual protocol

Seller publishes $X_H := x * H$

Seller:
* creates ranodm Nonce $R_0 := r_0 * H$, $R_1 := r_1 * H$ and sends it to the Buyer

Buyer:
* Gets random blinding factor $t_{0_0}$, $t_{0_1}$, $t_{1_0}$, $t_{1_1}$
* computes following items
  * blinded nonce
    * $R'_0 := R_0 + t_{0_1} * H + t_{0_1} * X_H$
    * $R'_1 := R_1 + t_{1_0} * H + t_{1_1} * X_H$
  * challenge
    * $c_0 := Hash(X_H || R'_0 || m)$
    * $c_1 := Hash(X_H || R'_1 || m)$
  * blinded challenge
    * $c'_0 := c_0 + t_{0_1}$
    * $c'_1 := c_1 + t_{1_1}$
  * And send blinded challenges to the Seller.

Seller:
* Get random i := {0, 1}
* computes $s := k_i + c'_i * x$
* Send following items
  * $s * G$
  * $PK\{(s, s_0, s_1) : s = s_0 \lor s = s_1 \}$

(The point here is that the Buyer can not know which he is going to receive $s_0$ or $s_1$)

Buyer:
* Make a payment offer with $s * G$

Seller:
* Receive payment by revaling $s$

Buyer:
* check if $s = s_0 * H$, $s = s_1 * H$ and unblind accordingly.


# Pay for partially-blind-signature

## Motivation

TODO:
* Anonymous credential is important building block



## General outline

### sell multiple discrete logarithm at once

When we want to sell several DL (say, x, y) atomically at once, we cannot just make a payment offer for each of them independently (even if we make the payment in the same AMP). Unlnke regular payment, the seller has a good reason to take only one payment and ignore others. So the seller first send some value computed from the DL he wants to sell, (e.g. r = x + y) and send it to the buyer along the proof of the relation so that if the buyer were able to know one of those values, he will be able to know every value he wants to purchase.

In this case send $r$ and $PK\{(x, y) : r = x + y \}$, then he can just purchase x and will be able to know y from $r - x$




## Protocol

The protocol is based on those of [Abe and Okamoto](https://www.iacr.org/archive/crypto2000/18800272/18800272.pdf)


### Preliminaties

besides the preliminaries defined in above, we define in addition

* $HashToG$ ... Hash function which hashes arbitrary length string into group element which no one knows its DL
* $Info$ ... Data previously shared with Buyer and the Seller


### Actual protocol

Seller:
* creates random variables $u$, $s$.
* get $Z := HashToG(Info)$
* create nonce point $R_a := G * u$, $R_b := G * s + Z * d$
* send nonce points ($R_a$, $R_b$) to the buyer

Buyer:
* Creates random blinding factors $t_1$, $t_2$, $t_3$, $t_4$
* get $Z := HashToG(info)$
* compute blinded nonces
  * $R_{\alpha} := R_a + G * t_1 + X * t_2$
  * $R_{\beta} := R_b + G * t_3 + Z * t_4$
* And blinded challenge
  * $\epsilon := Hash(R_{\alpha} || R_{\beta} || Z || msg)$
  * $e := \epsilon - t_2 - t_4$

Signer
* Computes
  * $c := e - d$
  * $r := u - cx$
  * $p = c + r$
* Send following items to the Buyer
  * $p$
  * $r * G$
  * $PK\{ (r, c, s, d) : c = e - d \land r = u - cx \}$
Buyer
* Make a payment offer with $r * G$, $c * G$, $s * G$, $d * G$






# Pay-for-Anonymous-credential


The anonymous credential scheme we use for this description is [Anonymous credentials light (ACL)](https://core.ac.uk/download/pdf/193377167.pdf)

Which is basically a partially-blind-signature-for-potentially-encrypted-attributes.

By extending the pay for blind-signature scheme we have described so far, we can construct the protocol to atomically exchange lightning BTC and the anonymous credential.



## Open Problems

* Must prove security formally.
* Extend the scheme into MAC based construction such as [KVAC](https://eprint.iacr.org/2013/516.pdf) which is more important for commercial usage of anonymous credential and thus fits well to LN-based payment.